In [ ]:
# Import packages

import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Activation, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import mean_squared_error
from scipy import stats
from math import sqrt
from keras.models import load_model
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
from tqdm.notebook import tqdm

In [ ]:
# Define function

def look_back(X, a):
    X_lb = np.zeros((len(X)- 29*a , a, 12))
    for i in range(len(X) - 29 * a):
        for j in range(a):
            X_lb[i, j] = X[i+(j*29)]
    X_lb = X_lb.reshape(int(len(X)/29) - a, 29, a, 12)
    Y_lb = X[a*29:, 7]
    Y_lb = Y_lb.reshape(int(len(X)/29) - a, 29, 1)
    return X_lb, Y_lb

def division(data):
    train_size = int(len(data)*0.6)
    val_size = int(len(data)*0.8)
    data_train = data[0:train_size]
    data_val = data[train_size:val_size]
    data_test = data[val_size:len(data)]
    return data_train, data_val, data_test

In [ ]:
# create the data about best time step 5

df = pd.read_csv('./data.csv', encoding='ms949')

ts = 5

train = df[:641*29]                    
validation = df[len(train):1006*29]    
test = df[len(train)+len(validation):] 

X_train, Y_train = look_back(train.values, ts)
Y_train = Y_train.reshape(len(Y_train), 29, 1, 1)

X_val, Y_val = look_back(validation.values, ts)
Y_val = Y_val.reshape(len(Y_val), 29, 1, 1)

X_test, Y_test = look_back(test.values, ts)
Y_test = Y_test.reshape(len(Y_test), 29, 1, 1)

In [ ]:
# Names of water-quality stations

stations = ['Goryeong','Naju', 'Nam River', 'Neungseo', 'Dalcheon', 'Lake Daecheong', 'Dogae', 'Lake Dongbok','Bokhacheon', 'Sinam', 'Andong Dam downstream', 'Yangpyeong', 'Yeoju', 'Lake Okjeong', 'Lake Yongdam', 'Yongbong', 'Uchi', 'Yugucheon', 'Lake Uiam', 'Jang-gye', 'Jeokpo', 'Lake Juam', 'Jiseokcheon', 'Cheongam', 'Chilgok', 'Lake Tamjin', 'Poongyang', 'Hyeondo', 'Hoesang']

In [ ]:
rmse_lst = []
r2_lst = []

for i in tqdm(range(0, 29, 1)):
    stst = datetime.datetime.now()
    
    trainX, valX, testX = X_train[:, i, :, :], X_val[:, i, :, :], X_test[:, i, :, :]
    trainY, valY, testY = Y_train[:, i, :, :].reshape(-1), Y_val[:, i, :, :].reshape(-1), Y_test[:, i, :, :].reshape(-1)
    
    early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 60, mode = 'min')
    mc = ModelCheckpoint('./LSTM_{}.h5'.format(stations[i]), 
                         monitor='val_loss', 
                         mode='min', 
                         save_best_only=True)
    
    model = Sequential() 
    model.add(LSTM(256, input_shape = (trainX.shape[1], trainX.shape[2]), return_sequences = True))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(LSTM(256, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(LSTM(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(LSTM(64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(LSTM(32, return_sequences=False))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1))
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    print("### Model training : time step_{} ###".format(ts))
    print('Start training :', datetime.datetime.now())
    model.fit(trainX, trainY,
              epochs=1000, 
              batch_size=8, 
              validation_data=(valX, valY), 
              callbacks=[early_stopping, mc],
              verbose=0)
    print('End training :', datetime.datetime.now())
    
    y_predicted = model.predict(testX)
    y_predicted = y_predicted.reshape(-1, 1).astype('float32')
    y_observed = testY.reshape(-1, 1).astype('float32')
    

    raw= {'Observed': list(y_observed), 'Predicted': list(y_predicted)}
    rr = pd.DataFrame(raw)
    reg = sm.OLS.from_formula("Observed ~ Predicted",rr).fit()


    try:
        rmse = round(sqrt(mean_squared_error(y_predicted, )), 3)
        r2 = round(reg.rsquared, 3)
    except ValueError:
        pass
    
    rmse_lst.append(rmse)
    r2_lst.append(r2)
    
    print('RMSE :', rmse)
    print('R-squared :', r2)
    print("---------------------------------\n")
    
    del model




    y_predicted = model.predict(testX)
    y_predicted = y_predicted.reshape(-1, 1).astype('float32')
    y_observed = testY.reshape(-1, 1).astype('float32')
    
    raw= {'Observed': list(y_observed), 'Predicted': list(y_predicted)}
    rr = pd.DataFrame(raw)
    reg = sm.OLS.from_formula("Observed ~ Predicted",rr).fit()


    try:
        rmse = round(sqrt(mean_squared_error(y_observed, y_predicted)), 3)
        r2 = round(reg.rsquared, 3)
    except ValueError:
        pass
    
    rmse_lst.append(rmse)
    r2_lst.append(r2)
    
    print('RMSE :', rmse)
    print('R-squared :', r2)
    print("---------------------------------\n")
    
    del model